In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

framer = pd.DataFrame()

# Turnover (Rate of Appearance (Ra)) Function

In [10]:
#Function to calculate turnover based on Joanne Kelleher's Linear Regression Modeling
#It takes the Tracer, Natural/Background (unenriched plasma), 
#and Observed/Experimental/Infused (plasma after infusion)
#in the format of mass distribution vectors (m0, m1, m2, m3, etc...) as fractions that add up to 1
#Then infusion rate in the units mg/kg/min. is taken
#These are taken in the format of Python lists

def Turnover_Ra(Tracer, Natural, Observed, infusion_rate):
    
    #Convert lists from raw MPEs into np arrays
    
    Tracer_ms = np.array(Tracer)
    Natural_ms = np.array(Natural)
    Observed_ms = np.array(Observed)
    
    #Perform subtractions to account for background MS signal
    observed_natural = Observed_ms - Natural_ms
    tracer_natural = Tracer_ms - Natural_ms
    
    #Obtain slope of linear regression based on simple format of y=mx+b
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(tracer_natural, observed_natural)
    
    #slope is our "p", the fraction of the total flux into blood due to tracer
    
    #use the slope to calculate Turnover (Rate of Appearance, Ra)
    Ra = (infusion_rate/slope) - infusion_rate
    
    return Ra

#EXAMPLE FROM PYTHON DICTIONARY FILLED WITH DATA
#Glucose_Ra_Mouse1 = Turnover_Ra(Glucose_Tracer, dict_of_plasma_metabolites['glucose'].iloc[:,2].tolist(), 
#                                dict_of_plasma_metabolites['glucose'].iloc[:,3].tolist(), glucose_infusion_rate)

In [21]:
Glucose_Ra_Mouse1 = Turnover_Ra(Glucose_Tracer, dict_of_plasma_metabolites['glucose'].iloc[:,2].tolist(), 
                                dict_of_plasma_metabolites['glucose'].iloc[:,3].tolist(), glucose_infusion_rate)

In [22]:
Glucose_Ra_Mouse1

5.686970507605056

# Plasma tracer contribution to tissue calculations

In [11]:
#This function takes a numpy array of the MDV of any nutrient and returns the
#atom percent enrichment (APE), or the fraction of the molecule that is actually labelled
#The first value in the MDV is multiplied by 0, so the m0 contribution of the molecule
#does not count towards the APE

#This equation is from Text Box 4 of Fernandez-Garcia et al 2020, DOI: 10.1016/j.tibs.2019.12.002

def FractionalAPE(NutrientMDV):
    carbons = np.arange(len(NutrientMDV))
    APE = sum(NutrientMDV*carbons)/(len(NutrientMDV)-1)
    return APE

# Define Tracer Infusion Rate

In [12]:
#define tracer infusion rate in mg/kg/min.

glucose_infusion_rate = 1 #* (25/body_weight)
leucine_infusion_rate = 0.1

#Be sure to correct for individual animal body weight if infusion rate was calculated based on an
#average 25g mouse

In [13]:
#Define Theoretical Tracer Enrichment MPEs

#U13C-Glucose [m0,m1,m2,m3,m4,m5,m6]
Glucose_Tracer = [0,0,0,0,0,0,1]

#1-13C-Leucine
Leucine_Tracer = [0,1,0,0,0,0,0]

# Load in Data

In [3]:
#Read in data for turnover calculations

data = "/Users/brooksleitner/Desktop/Python/PerryLabData/Sepsis/acutemetabolomics/WonAnalyses/Summary_fractional_labeling.xlsx"
plasma = pd.read_excel(data, sheet_name=0, header=0)
tissue = pd.read_excel(data, sheet_name=1, header=0)

In [4]:
plasma

,Compound,C_Label,20211129_1_Plasma,20211129_3_Plasma,20211129_2_Plasma,20211129_4_Plasma,20211129_11_Plasma,20211129_12_Plasma,20211129_13_Plasma,20211129_14_Plasma,20211129_21_Plasma,20211129_22_Plasma,20211129_23_Plasma,20211129_24_Plasma
0,pyruvate,0,0.998188,0.951463,0.996708,0.966752,0.998052,0.963660,0.998673,0.975013,0.998461,0.997331,0.997815,0.998328
1,pyruvate,1,0.001046,0.004610,0.002846,0.005648,0.001834,0.006661,0.001317,0.006105,0.001326,0.002472,0.001980,0.001573
2,pyruvate,2,0.000070,0.004096,0.000000,0.003156,0.000000,0.003173,0.000000,0.002198,0.000044,0.000000,0.000000,0.000000
3,pyruvate,3,0.000697,0.039831,0.000446,0.024444,0.000115,0.026506,0.000010,0.016684,0.000169,0.000196,0.000204,0.000099
4,glutamate,0,1.000000,0.996325,1.000000,0.993862,1.000000,0.987268,1.000000,0.986262,1.000000,0.994825,0.999056,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,acetoacetate,4,0.000000,0.016049,0.004251,0.005510,0.009178,0.005966,0.009827,0.007685,0.000000,0.006832,0.003321,0.013679
145,alanine,0,0.996386,0.968932,0.998050,0.982045,0.998511,0.975917,0.995977,0.983699,0.993066,0.998441,0.997401,0.996300
146,alanine,1,0.002274,0.002123,0.000000,0.001172,0.000000,0.005172,0.000000,0.000000,0.003248,0.000000,0.000245,0.000111
147,alanine,2,0.001340,0.003430,0.001501,0.002272,0.001489,0.003875,0.004023,0.004604,0.003685,0.001559,0.002354,0.003589


In [5]:
#Create a list of all unique measured metabolites

plasma_metabolites = plasma['Compound'].unique().tolist()
tissue_metabolites = tissue['Compound'].unique().tolist()

#Create a new dataframe for each individual metabolite

dict_of_plasma_metabolites = dict(tuple(plasma.groupby("Compound")))
dict_of_tissue_metabolites = dict(tuple(tissue.groupby("Compound")))


In [8]:
dict_of_plasma_metabolites['acetoacetate']

,Compound,C_Label,20211129_1_Plasma,20211129_3_Plasma,20211129_2_Plasma,20211129_4_Plasma,20211129_11_Plasma,20211129_12_Plasma,20211129_13_Plasma,20211129_14_Plasma,20211129_21_Plasma,20211129_22_Plasma,20211129_23_Plasma,20211129_24_Plasma
140,acetoacetate,0,1.0,0.972516,0.967076,0.969571,0.958681,0.993321,0.990173,0.978878,1.0,0.984712,0.986672,0.933242
141,acetoacetate,1,0.0,0.011435,0.028674,0.024918,0.032141,0.000713,0.000000,0.013437,0.0,0.008456,0.010008,0.053079
142,acetoacetate,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
143,acetoacetate,3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
144,acetoacetate,4,0.0,0.016049,0.004251,0.005510,0.009178,0.005966,0.009827,0.007685,0.0,0.006832,0.003321,0.013679


In [ ]:
#for FluxFraction, Turnover, and FractionalAPE we need lists of MDVs for each subject (column) for each metabolite
#for this, we will loop through every column of every dataframe

enrichment = 0
for value in dict_of_plasma_metabolites:
    enrichment += value
    


In [7]:
#Samples 1 and 2 are pre-U13C-glucose infusion CLP (background)
#Samples 3 and 4 are 120 min U13C-glucose infusion CLP
#Samples 11 and 13 are pre-U13C-glucose infusion Sham (background)
#Samples 12 and 14 are 120 min U13C-glucose infusion Sham
#Samples 21 and 23 are 120 minute 1-13C leucine infusion CLP
#Samples 22 and 24 are 120 minute 1-13C leucine infusion Sham

In [10]:
#Samples 1 and 3 are from the same animal

p_glucose_CLP_1 = FluxFraction(Glucose_Tracer, plasma_glucose['20211129_1_Plasma'].tolist(), 
                               plasma_glucose['20211129_3_Plasma'].tolist())
Glucose_Ra_CLP_1 = Turnover(glucose_infusion_rate, p_glucose_CLP_1)

#Samples 2 and 4 are from the same animal
p_glucose_CLP_2 = FluxFraction(Glucose_Tracer, plasma_glucose['20211129_2_Plasma'].tolist(), 
                               plasma_glucose['20211129_4_Plasma'].tolist())
Glucose_Ra_CLP_2 = Turnover(glucose_infusion_rate, p_glucose_CLP_2)

#11 and 12 are same Sham mouse

p_glucose_Sham_1 = FluxFraction(Glucose_Tracer, plasma_glucose['20211129_11_Plasma'].tolist(), 
                               plasma_glucose['20211129_12_Plasma'].tolist())
Glucose_Ra_Sham_1 = Turnover(glucose_infusion_rate, p_glucose_Sham_1)

#13 and 14 are same Sham mouse

p_glucose_Sham_2 = FluxFraction(Glucose_Tracer, plasma_glucose['20211129_13_Plasma'].tolist(), 
                               plasma_glucose['20211129_14_Plasma'].tolist())
Glucose_Ra_Sham_2 = Turnover(glucose_infusion_rate, p_glucose_Sham_2)


In [11]:
print(Glucose_Ra_CLP_1, Glucose_Ra_CLP_2, Glucose_Ra_Sham_1, Glucose_Ra_Sham_2)

5.686970507605056 9.584843497367551 23.528869166056946 39.26873790527808


In [12]:
#Using background from U13C-glucose mice for leucine calculations

#Samples 21 and 23 are 120 minute 1-13C leucine infusion CLP

p_leucine_CLP_1 = FluxFraction(Leucine_Tracer, plasma_leucine['20211129_1_Plasma'].tolist(), 
                               plasma_leucine['20211129_21_Plasma'].tolist())
Leucine_Ra_CLP_1 = Turnover(leucine_infusion_rate, p_leucine_CLP_1)

p_leucine_CLP_2 = FluxFraction(Leucine_Tracer, plasma_leucine['20211129_1_Plasma'].tolist(), 
                               plasma_leucine['20211129_23_Plasma'].tolist())
Leucine_Ra_CLP_2 = Turnover(leucine_infusion_rate, p_leucine_CLP_2)


#Samples 22 and 24 are 120 minute 1-13C leucine infusion Sham

p_leucine_Sham_1 = FluxFraction(Leucine_Tracer, plasma_leucine['20211129_1_Plasma'].tolist(), 
                               plasma_leucine['20211129_22_Plasma'].tolist())
Leucine_Ra_Sham_1 = Turnover(leucine_infusion_rate, p_leucine_Sham_1)

p_leucine_Sham_2 = FluxFraction(Leucine_Tracer, plasma_leucine['20211129_1_Plasma'].tolist(), 
                               plasma_leucine['20211129_24_Plasma'].tolist())
Leucine_Ra_Sham_2 = Turnover(leucine_infusion_rate, p_leucine_Sham_2)


In [13]:
print(Leucine_Ra_CLP_1, Leucine_Ra_CLP_2, Leucine_Ra_Sham_1, Leucine_Ra_Sham_2)

1.7666731938659916 1.9660382967162278 1.8628213997529426 1.9078410448914882
